# MTL

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'.csv')
output_names = ['Electricity Consumption', 'Humidity Closer to Evaporator (%)', 'Temperature Closer to Evaporator (C)']

from MTL_FDCS_2_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1800, 88)
Shape of y_train_normalized: (1800, 3)
Shape of X_val_normalized: (336, 88)
Shape of y_val_normalized: (336, 3)
Shape of X_test_normalized: (336, 88)
Shape of y_test_normalized: (336, 3)
Shape of X_train_val_normalized: (2136, 88)
Shape of y_train_val_normalized: (2136, 3)


In [2]:
from FDCS_2_MLT_FS import evaluate_model
final_df = evaluate_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
final_df

2024-03-07 15:58:58.668181: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 15:58:58.670254: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 15:58:58.710658: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 15:58:58.711669: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 15:58:59.420824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100
12/12 [==============================] - 1s 5ms/step - loss: 0.4192 - dense_2_loss: 0.0597 - dense_3_loss: 0.2203 - dense_4_loss: 0.1392 - dense_2_mse: 0.0597 - dense_3_mse: 0.2203 - dense_4_mse: 0.1392
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 0.1080 - dense_2_loss: 0.0293 - dense_3_loss: 0.0559 - dense_4_loss: 0.0228 - dense_2_mse: 0.0293 - dense_3_mse: 0.0559 - dense_4_mse: 0.0228
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 0.0818 - dense_2_loss: 0.0271 - dense_3_loss: 0.0344 - dense_4_loss: 0.0202 - dense_2_mse: 0.0271 - dense_3_mse: 0.0344 - dense_4_mse: 0.0202
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 0.0667 - dense_2_loss: 0.0243 - dense_3_loss: 0.0268 - dense_4_loss: 0.0157 - dense_2_mse: 0.0243 - dense_3_mse: 0.0268 - dense_4_mse: 0.0157
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 0.0602 - dense_2_loss: 0.0224 - dense_3_loss: 0.0224 - dense_4_loss

,True,Pred,Model,Output,Feature_Selection
0,0.487435,0.434845,MTL,Electricity Consumption,Without
1,0.536867,0.412461,MTL,Electricity Consumption,Without
2,0.434658,0.387870,MTL,Electricity Consumption,Without
3,0.542578,0.364308,MTL,Electricity Consumption,Without
4,0.320490,0.338371,MTL,Electricity Consumption,Without
...,...,...,...,...,...
9067,0.634877,0.558210,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9068,0.204360,0.513596,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9069,0.136240,0.493140,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9070,0.226158,0.517201,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each combination of Feature_Selection method and Output
grouped = final_df.groupby(['Feature_Selection', 'Output'])
metrics = []

for (method, output), group in grouped:
    true_values = group['True']
    pred_values = group['Pred']
    mae = mean_absolute_error(true_values, pred_values)
    rmse = np.sqrt(mean_squared_error(true_values, pred_values))
    metrics.append({'Method': method, 'Output': output, 'MAE': mae, 'RMSE': rmse})

metric_df = pd.DataFrame(metrics). sort_values(by='MAE')
metric_df


,Method,Output,MAE,RMSE
4,Embedded - Lasso,Humidity Closer to Evaporator (%),0.108665,0.142709
10,Filter - Correlation,Humidity Closer to Evaporator (%),0.117535,0.150686
11,Filter - Correlation,Temperature Closer to Evaporator (C),0.119965,0.174149
0,Embedded - ElasticNet,Electricity Consumption,0.121420,0.158314
5,Embedded - Lasso,Temperature Closer to Evaporator (C),0.123077,0.165333
3,Embedded - Lasso,Electricity Consumption,0.126078,0.160209
16,Hybrid - Filter (SelectKBest) + Embedded (RF),Humidity Closer to Evaporator (%),0.126742,0.155119
17,Hybrid - Filter (SelectKBest) + Embedded (RF),Temperature Closer to Evaporator (C),0.128245,0.174663
26,Wrapper - RFE (LinearRegression),Temperature Closer to Evaporator (C),0.128509,0.181361
1,Embedded - ElasticNet,Humidity Closer to Evaporator (%),0.129398,0.165124


In [4]:
import pandas as pd
import joblib
# Load the saved scalers
scalers_y_loaded = joblib.load('scalers_y_multi_targets.joblib')
targets = output_names

# Update the loop to filter rows by target and then apply the inverse transform
for target in targets:
    # Filter rows for the current target
    target_rows = final_df[final_df['Output'] == target]
    
    # Check if there are any rows to update to avoid KeyErrors
    if not target_rows.empty:
        # Apply inverse transformation to the true values for the current target
        true_values_transformed = scalers_y_loaded[target].inverse_transform(target_rows[['True']].values)
        pred_values_transformed = scalers_y_loaded[target].inverse_transform(target_rows[['Pred']].values)
        
        # Update the DataFrame with the inverse transformed values
        final_df.loc[final_df['Output'] == target, 'True'] = true_values_transformed
        final_df.loc[final_df['Output'] == target, 'Pred'] = pred_values_transformed

# Now final_df should have the true and predicted values back in their original scale
final_df.to_csv('final_df_FDCS_2_MTL.csv')
final_df

,True,Pred,Model,Output,Feature_Selection
0,1638.49,1461.708740,MTL,Electricity Consumption,Without
1,1804.65,1386.466797,MTL,Electricity Consumption,Without
2,1461.08,1303.805664,MTL,Electricity Consumption,Without
3,1823.85,1224.603760,MTL,Electricity Consumption,Without
4,1077.31,1137.415527,MTL,Electricity Consumption,Without
...,...,...,...,...,...
9067,10.33,10.048632,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9068,8.75,9.884895,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9069,8.50,9.809823,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9070,8.83,9.898127,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)


## GridSearch/Keras Tuner

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'.csv')
output_names = ['Electricity Consumption', 'Humidity Closer to Evaporator (%)', 'Temperature Closer to Evaporator (C)']

from MTL_FDCS_2_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1272, 85)
Shape of y_train: (1272, 3)
Shape of X_val_normalized: (336, 85)
Shape of y_val: (336, 3)
Shape of X_test_normalized: (336, 85)
Shape of y_test: (336, 3)
Shape of X_train_val_normalized: (1608, 85)
Shape of y_train_val: (1608, 3)


In [2]:
from FDCS_1_MLT_GridSearch import evaluate_model_with_tuner

# Call the function
best_hyperparams, metrics_df, pred_true_TestSet = evaluate_model_with_tuner(
    X_train_normalized,
    y_train_normalized,
    X_val_normalized,
    y_val_normalized,
    output_names
)


metrics_df


Trial 10 Complete [00h 00m 12s]
val_loss: 0.3243689760565758

Best val_loss So Far: 0.11876774206757545
Total elapsed time: 00h 01m 58s
Epoch 1/100
40/40 [==============================] - 1s 3ms/step - loss: 0.2826 - dense_7_loss: 0.0938 - dense_8_loss: 0.1051 - dense_9_loss: 0.0836 - dense_7_mse: 0.0938 - dense_8_mse: 0.1051 - dense_9_mse: 0.0836
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 0.1955 - dense_7_loss: 0.0657 - dense_8_loss: 0.0657 - dense_9_loss: 0.0641 - dense_7_mse: 0.0657 - dense_8_mse: 0.0657 - dense_9_mse: 0.0641
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 0.1865 - dense_7_loss: 0.0619 - dense_8_loss: 0.0629 - dense_9_loss: 0.0617 - dense_7_mse: 0.0619 - dense_8_mse: 0.0629 - dense_9_mse: 0.0617
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 0.1828 - dense_7_loss: 0.0606 - dense_8_loss: 0.0616 - dense_9_loss: 0.0606 - dense_7_mse: 0.0606 - dense_8_mse: 0.0616 - dense_9_mse: 0.0606
Epoc

,Fold,Set,Model,Output,MAE,RMSE
0,Final,Test,MTL,Electricity Consumption,0.187332,0.243134
1,Final,Test,MTL,Humidity Closer to Evaporator (%),0.129914,0.155930
2,Final,Test,MTL,Temperature Closer to Evaporator (C),0.127582,0.145156


In [3]:
best_hyperparams

{'units_1': 10, 'units_2': 10, 'learning_rate': 0.001954220611224397}

In [4]:
pred_true_TestSet

,True,Pred,Model,Output
0,0.004761,0.418643,MTL,Electricity Consumption
1,0.002706,0.435705,MTL,Electricity Consumption
2,0.003977,0.445145,MTL,Electricity Consumption
3,0.005189,0.439359,MTL,Electricity Consumption
4,0.005006,0.413817,MTL,Electricity Consumption
...,...,...,...,...
1003,0.326146,0.354285,MTL,Temperature Closer to Evaporator (C)
1004,0.303235,0.343624,MTL,Temperature Closer to Evaporator (C)
1005,0.235849,0.362303,MTL,Temperature Closer to Evaporator (C)
1006,0.235849,0.294750,MTL,Temperature Closer to Evaporator (C)


In [5]:
pred_true_TestSet.to_csv('FDCS_1_MTL_GridSearch.csv')